You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [3]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [4]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [5]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [7]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [8]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [9]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [12]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 
Title: The Future of Artificial Intelligence: Latest Trends and Key Players

Introduction:
- Brief overview of the rapid advancements in artificial intelligence technology.
- Mention of the key players in the industry and their contributions.
- Introduction to the target audience and why this topic is relevant to them.
- SEO keywords: artificial intelligence trends, AI key players, future of AI.

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine learning and deep learning adva

I now can give a great answer

Final Answer: 

# The Future of Artificial Intelligence: Latest Trends and Key Players

**Introduction:**
Artificial Intelligence (AI) has been rapidly advancing in recent years, reshaping various industries and revolutionizing the way we interact with technology. Key players in the AI industry, such as Google, Microsoft, and Amazon, have been at the forefront of driving innovation and pushing the boundaries of what AI can achieve. In this blog post, we will explore the latest trends in artificial intelligence and delve into the key players shaping the future of AI.

**Latest Trends in Artificial Intelligence:**
One of the most significant trends in AI is the advancements in machine learning and deep learning algorithms, enabling machines to learn from data and make predictions without explicit programming. Natural language processing and sentiment analysis have also seen significant progress, allowing AI systems to understand and generate human language 

- Display the results of your execution as markdown in the notebook.

In [13]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Latest Trends and Key Players

**Introduction:**
Artificial Intelligence (AI) has been rapidly advancing in recent years, reshaping various industries and revolutionizing the way we interact with technology. Key players in the AI industry, such as Google, Microsoft, and Amazon, have been at the forefront of driving innovation and pushing the boundaries of what AI can achieve. In this blog post, we will explore the latest trends in artificial intelligence and delve into the key players shaping the future of AI.

**Latest Trends in Artificial Intelligence:**
One of the most significant trends in AI is the advancements in machine learning and deep learning algorithms, enabling machines to learn from data and make predictions without explicit programming. Natural language processing and sentiment analysis have also seen significant progress, allowing AI systems to understand and generate human language with increasing accuracy. AI is increasingly being applied in healthcare, finance, and other industries, transforming processes and improving efficiency. However, as AI continues to evolve, ethical considerations in its development have become more prominent, raising questions about privacy, bias, and accountability.

**Key Players in the AI Industry:**
Tech giants like Google, Microsoft, and Amazon are leading the way in AI research and development, leveraging their resources and expertise to drive innovation. In addition to established companies, startups are disrupting the AI space with fresh ideas and agile approaches. Research institutions and academia play a crucial role in advancing AI technologies, collaborating with industry partners to push the boundaries of what AI can achieve. Influential AI experts and thought leaders are also shaping the conversation around AI, sharing insights and driving the industry forward.

**Noteworthy News in Artificial Intelligence:**
Recent breakthroughs in AI research, such as advancements in computer vision and reinforcement learning, have captured the industry's attention and sparked excitement about the possibilities of AI. Industry collaborations and partnerships are fostering innovation and accelerating the pace of AI development. Regulatory developments are also impacting AI adoption, with governments and organizations working to establish guidelines and standards for ethical AI use. Case studies of successful AI implementations showcase the tangible benefits of adopting AI technologies in various sectors, from healthcare to finance.

**Conclusion:**
As AI continues to evolve and shape the future of technology, staying informed about the latest trends and key players in the industry is essential for tech enthusiasts, industry professionals, and business leaders. By understanding the advancements in AI technologies and the contributions of key players, individuals can make informed decisions and stay ahead of the curve in the rapidly evolving AI landscape. Subscribe for regular updates on AI trends and news to stay informed and engaged with the exciting developments in artificial intelligence.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [16]:
topic = "Nha Trang"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Nha Trang.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan for Blog Article on Nha Trang

Title: Exploring the Beauty of Nha Trang: A Guide to the Latest Trends and Key Players

Outline:
I. Introduction
- Brief overview of Nha Trang as a popular tourist destination in Vietnam
- Mention of the latest trends and key players in the area

II. Latest Trends in Nha Trang
- Rise of eco-friendly and sustainable tourism practices
- Growing popularity of wellness retreats and spa resorts
- Emergence of local craft markets and artisanal shops

III. Key Players in Nha Trang
- Le

I now can give a great answer

Final Answer:

# Exploring the Beauty of Nha Trang: A Guide to the Latest Trends and Key Players

## Introduction
Nha Trang, a picturesque coastal city in Vietnam, has long been known for its stunning beaches and vibrant culture, making it a popular tourist destination. In recent years, Nha Trang has seen a surge in eco-friendly and sustainable tourism practices, as well as an increase in wellness retreats and spa resorts. This article delves into the latest trends and key players in Nha Trang, offering insights for travelers looking to explore this hidden gem.

## Latest Trends in Nha Trang
One of the notable trends in Nha Trang is the rise of eco-friendly and sustainable tourism practices. From beach clean-up initiatives to eco-friendly accommodations, the city is making strides towards preserving its natural beauty for future generations. Additionally, wellness retreats and spa resorts have become increasingly popular, offering travelers a chance to re

In [17]:
Markdown(result)

# Exploring the Beauty of Nha Trang: A Guide to the Latest Trends and Key Players

## Introduction
Nha Trang, a picturesque coastal city in Vietnam, has long been known for its stunning beaches and vibrant culture, making it a popular tourist destination. In recent years, Nha Trang has seen a surge in eco-friendly and sustainable tourism practices, as well as an increase in wellness retreats and spa resorts. This article delves into the latest trends and key players in Nha Trang, offering insights for travelers looking to explore this hidden gem.

## Latest Trends in Nha Trang
One of the notable trends in Nha Trang is the rise of eco-friendly and sustainable tourism practices. From beach clean-up initiatives to eco-friendly accommodations, the city is making strides towards preserving its natural beauty for future generations. Additionally, wellness retreats and spa resorts have become increasingly popular, offering travelers a chance to rejuvenate and unwind in a tranquil setting. Nha Trang has also seen the emergence of local craft markets and artisanal shops, providing visitors with a chance to support local artisans and take home unique souvenirs.

## Key Players in Nha Trang
Nha Trang is home to leading hotels and resorts that offer unique and luxurious experiences for guests. From beachfront villas to boutique hotels, travelers are spoiled for choice when it comes to accommodations in Nha Trang. The city also boasts top-rated restaurants that showcase authentic Vietnamese cuisine, tantalizing the taste buds of foodies from around the world. Local tour operators play a crucial role in providing immersive cultural experiences, giving visitors a chance to explore the rich history and traditions of Nha Trang.

## Target Audience
This article is tailored to travel enthusiasts seeking off-the-beaten-path destinations, wellness seekers looking for rejuvenating experiences, and foodies eager to explore the local culinary scene. Whether you're interested in eco-friendly tourism, luxurious accommodations, or authentic Vietnamese cuisine, Nha Trang has something to offer for every type of traveler.

## SEO Keywords
- Nha Trang travel guide
- Eco-friendly tourism in Nha Trang
- Best hotels in Nha Trang
- Authentic Vietnamese cuisine Nha Trang

## Call to Action
As you plan your trip to Nha Trang, consider exploring the latest trends and key players mentioned in this article. Check out reputable tour operators and accommodation options to make the most of your visit. Share your experience and leave feedback for fellow travelers who are looking to discover the beauty of Nha Trang.

By following this guide, you'll be well-equipped to embark on a memorable journey to Nha Trang, where the beauty of the city and the warmth of its people await you. Happy travels!

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).